In [1]:
using VerilogWriter

┌ Info: Precompiling VerilogWriter [609488a3-1988-4612-be37-e0d5b22d8f97]
└ @ Base loading.jl:1423


## Examples in Documents

In [2]:
let
    prs = @parameters (dummy = 10)

    ps = @ports (
        @in CLK, RST, sig1, sig2;
        @in 8 din;
        @out @reg 8 dout
    )

    ds = @decls (
        @reg 8 dbuf
    )

    proc = @always (
        if sig1
            dbuf <= din 
        elseif sig2 
            dout <= dbuf
        else
            dout <= ~din 
        end
    )

    mymod = Vmodule(
        "mymodule",
        prs,
        ps,
        ds,
        Assign[],
        [autoreset(proc)]
    )

    vshow(mymod)
end

module mymodule #(
    parameter dummy = 10
)(
    input CLK,
    input RST,
    input sig1,
    input sig2,
    input [7:0] din,
    output reg [7:0] dout
);
    
    reg [7:0] dbuf;

    always_ff @( posedge CLK ) begin
        if (RST) begin
            dbuf <= 0;
            dout <= 0;
        end else begin
            if (sig1) begin
                dbuf <= din;
            end else if (sig2) begin
                dout <= dbuf;
            end else begin
                dout <= ~din;
            end
        end
    end
endmodule
type: Vmodule


In [3]:
function main()
    # AXI master, no `multiple outstanding addresses`
    prms = @parameters (
        DWID = 32
    )

    mawport = @ports (
        @out awvalid;
        @in awready;
        @out 32 awaddr;
        @out 3 awprot
    )
    mwport = @ports (
        @out wvalid;
        @in wready;
        @out DWID wdata;
        @out DWID >> 3 wstrb
    )
    mbport = @ports (
        @in bvalid;
        @out bready;
        @in 2 bresp
    )

    mwportall = declmerge(mawport, mwport, mbport)
    # vshow(mwportall)
    wfsm = @FSM wstate widle, waready, wdready, wready
    
    i2a0 = @wireexpr awvalid & awready 
    i2d0 = @wireexpr wvalid & wready 
    i2a = i2a0 & ~i2d0
    i2d = ~i2a0 & i2d0
    i2r = i2a0 & i2d0
    a2r = i2d0
    d2r = i2a0
    r2i = @wireexpr bvalid & bready

    transadd!(
        wfsm, 
        [
            (i2a, @tstate widle => waready),
            (i2d, @tstate widle => wdready),
            (i2r, @tstate widle => wready),
            (a2r, @tstate waready => wready),
            (d2r, @tstate wdready => wready),
            (r2i, @tstate wready => widle)
        ]
    )

    wfsmalways = autoreset(fsmconv(Ifcontent, wfsm))

    vshow(wfsmalways)
end

# @btime main()
@time main()
# main()

always_ff @( posedge CLK ) begin
    if (RST) begin
        wstate <= 0;
    end else begin
        case (wstate)
            widle: begin
                if (((awvalid & awready) & ~(wvalid & wready))) begin
                    wstate <= waready;
                end else if ((~(awvalid & awready) & (wvalid & wready))) begin
                    wstate <= wdready;
                end else if (((awvalid & awready) & (wvalid & wready))) begin
                    wstate <= wready;
                end
            end
                waready: begin
                if ((wvalid & wready)) begin
                    wstate <= wready;
                end
            end
                wdready: begin
                if ((awvalid & awready)) begin
                    wstate <= wready;
                end
            end
                wready: begin
                if ((bvalid & bready)) begin
                    wstate <= widle;
                end
            end
        endcase
    end
end
type

In [3]:
using BenchmarkTools

In [5]:
println("case (state)\n    suno: begin\n        x = y;\n    end\n        sdos: begin\n        y <= z;\n    end\nendcase")
println("case (state)\n    suno: begin\n        x = y;\n    end\n    sdos: begin\n        y <= z;\n    end\nendcase")

case (state)
    suno: begin
        x = y;
    end
        sdos: begin
        y <= z;
    end
endcase
case (state)
    suno: begin
        x = y;
    end
    sdos: begin
        y <= z;
    end
endcase
